### Figure 4 
- Sliding template during sleep deprivation
- Correlation across time window

### load data

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from subjects import boxplot_kw
import subjects
from subjects import sns_boxplot_kw,stat_kw
import pandas as pd
import seaborn as sns
from neuropy.plotting.dataviz import barplot
from neuropy.plotting import Fig
from scipy import stats
from statannotations.Annotator import Annotator

group = ['NSD','SD']
grpdata = subjects.GroupData()
pf_norm_tuning = grpdata.pf_norm_tuning


# replay examples
examples = grpdata.replay_examples

#replay rate
wcorr_dist = grpdata.replay_wcorr
# post replay score
post_df = grpdata.replay_post_score
# post_df = post_df[np.abs(post_df['velocity'])>100]
post_df['weighted_corr'] = post_df['weighted_corr'].abs()
colors_post = subjects.colors_sd(1)


fig = Fig(grid=(6, 6), hspace=0.45, wspace=0.5)
filename = subjects.figpath_sd / 'figure4'
caption = 'Place cells: \n(A) Place fields recorded during NSD and SD sessions.'

### replay examples

In [ ]:
subfig = fig.add_subfigure(fig.gs[1, 0:3])
axs = subfig.subplots(2, 8, sharey="row")

dt = 0.02*1000
pf_cmaps = ['Greys','Reds']
for i, g in enumerate(group):
    events = examples[g]["posteriors"]
    wcorr = examples[g]['wcorr']
    for i1, ev in enumerate(events):
        ev = np.apply_along_axis(
            np.convolve, axis=0, arr=ev, v=np.ones(2 * 2 + 1), mode="same"
        )
        nt = ev.shape[1]
        ax = axs[i,i1]
        ax.pcolormesh(ev, cmap=pf_cmaps[i], vmin=0, vmax=0.2,rasterized=True)
        ax.text(0.6,7,f'{wcorr[i1].round(2)}',color='#039be5',fontdict=dict(fontsize=6))
        ax.spines['right'].set_visible(True)
        ax.spines['top'].set_visible(True)
        ax.set_yticks([])
        ax.set_xticks([nt//2],[int(nt*dt)])
        ax.tick_params(length=0)


### wcorr sequence score 

In [ ]:
colors = subjects.colors_sd(1)
for g,grp in enumerate(['NSD','SD']):
    df = wcorr_dist[wcorr_dist['grp']==grp]

    ax = fig.subplot(fig.gs[g, 3])
    plot_kw =dict(data=df,x='zt',y='seq_score',ax=ax,color=colors[g]) 
    # del sns_boxplot_kw['palette']
    sns.boxplot(**plot_kw,**sns_boxplot_kw,width=0.7)

    #stats
    orders =df.zt.unique()
    pairs = [('PRE', _) for _ in orders[1:]]
    annotator = Annotator(pairs=pairs, **plot_kw, order=orders)
    annotator.configure(test="t-test_welch", **stat_kw)
    annotator.apply_and_annotate()
    annotator.reset_configuration()

    ax.set_xlabel('')
    ax.set_ylim(-2.5,10)
    ax.tick_params('x',rotation=30)
    ax.legend('',frameon=False)

### saving

In [ ]:
fig.savefig(filename,caption=caption)